# Getting started with Queues Demo

Fun with queues! We'll be exploring SQS by demo.

### Let's start with a Standard SQS Queue

Provides a distributed queue that is
* Simple to use
* Provides nearly unlimited throughput
* Runs as a highly available regional AWS Service

Considerations
* Message order is not gauranteed.
* Messages will be delivered at least once, but could be duplicated.
* If using AWS SageMaker, verify the notebook is using an IAM role with access to SQS.

In [ ]:
import time
import json
import boto3

sqs = boto3.resource('sqs')
queue = sqs.create_queue(QueueName="Demo-queue") # This will be a standard queue by default

print(queue)

### Let's Put Some Messages In the Queue

Let's add some messages and review the queue properties.

In [ ]:
queue = sqs.get_queue_by_name(QueueName='Demo-queue')

response = queue.send_message(
    MessageBody='http://example.com/input/falcon9.mp4'
)

print(json.dumps(response,indent=2))

In [ ]:
response = queue.send_message(
    MessageBody='http://example.com/input/curiousity.mp4'
)

print(json.dumps(response,indent=2))

### Let's check the queue properties

In [ ]:

print(json.dumps(queue.attributes,indent=2))

### Let's Read Messages from the Queue

Note: This consumer code could be running on a different process... or on another computer!

In [ ]:
import time
for iteration in range(0,5):
    
    messages = queue.receive_messages(MaxNumberOfMessages= 5, # Max number of messages to receive per poll
                                      WaitTimeSeconds= 4) # How long to wait for new messages!
    print("Iteration {0} of polling SQS. Got {1} messages this time.".format(
        iteration,len(messages)))
    
    for message in messages:
            
        # Simulate processing the message by printing    
        print ('\nGot Message!\nMessage id: {0} \nMD5 of body - {1}\nMessage Body {2}\nProcessed!\n'.format(
            message.message_id, message.md5_of_body, message.body))
        
        

        message.delete() # delete the message so no one else gets the work item
    


### Adding Custom Attributes to a Message

In [ ]:
# Message with Custom Attributes via http://boto3.readthedocs.io/en/latest/guide/sqs.html
# Custom attributes can be of type string, number, binary, or custom data types.

queue.send_message(MessageBody='Queues are fun', MessageAttributes={
    'MyCustomStringAttr': {
        'StringValue': '123456',
        'DataType': 'String'
    },
    'MyCustomNumberAttr' : {
        'StringValue' : '3.50',
        'DataType' : 'Number'
    }
})

In [ ]:
# Reading Custom Attributes
for iteration in range(0,5):

    # Read specific attributes or All attributes
    messages = queue.receive_messages(MessageAttributeNames=['All'],WaitTimeSeconds= 4)
    print('Iteration %s - polled queue and received %s message(s) received'%(iteration, len(messages)))
    for message in messages:
        # Get the custom author message attribute if it was set
        print('Message body: %s'%(message.body))

        if (message.message_attributes):
            for attr in message.message_attributes:
                print('Custom Attribute: %s - Value: %s'%(attr, message.message_attributes[attr]))
        else:
            print ('No custom attributes present.')

        # Let the queue know that the message is processed
        message.delete()

# Fifo Queue Example

[SQS FIFO (First-In-First-Out) queues](https://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/FIFO-queues.html) are designed to enhance messaging between applications when the order of operations and events is critical, or where duplicates can't be tolerated, for example:

- Ensure that user-entered commands are executed in the right order.
- Display the correct product price by sending price modifications in the right order.
- Prevent a student from enrolling in a course before registering for an account.

FIFO queues also provide exactly-once processing but have a limited number of transactions per second (TPS). However as of 12/21/2020 [High Throughput FIFO Queues are in preview](https://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/high-throughput-fifo.html).


### Create a Fifo Queue

In [ ]:
import boto3
sqs = boto3.resource('sqs', region_name='us-east-1')
fifo = sqs.create_queue(QueueName="demo-fifo-queue.fifo", Attributes={'FifoQueue':'true'})
print(fifo)

### Set Strategy for identifying duplicate messages based on Message Body Content

This is just one of several qays Fifo queues can try to deduplicate your messages


In [ ]:
response = fifo.set_attributes(Attributes={'ContentBasedDeduplication':'true'})
print(json.dumps(response,indent=2))

### Let's put in some messages

Remember fifo queues will filter out duplicate messages (for 5 minutes by default)
and preserve ordering within a Group Id

In order processing of messages. within a message group.
3 part message - 1 provision account , 2  provision resources, 3 trigger notification to customer

In [ ]:
fifo.send_message(MessageBody='Message #1',MessageGroupId='Group1')
fifo.send_message(MessageBody='Message #1',MessageGroupId='Group2')
fifo.send_message(MessageBody='Message #2',MessageGroupId='Group2')

fifo.send_message(MessageBody='Message #2',MessageGroupId='Group1')
fifo.send_message(MessageBody='Message #2',MessageGroupId='Group1')

fifo.send_message(MessageBody='Message #3',MessageGroupId='Group2')


### Let's get the messages back out of the queue


How many messages will we get using the selected deduplication strategy??
Does the content based deduplication strategy differentiate between message groups?
Let's find out.

PS: [High Throughput FIFO Queues (in preview)](https://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/high-throughput-fifo.html) will provide more choices around deduplication scope.

In [ ]:
import time
for iteration in range(0,10):
    
    messages = fifo.receive_messages(AttributeNames=['ALL'], MaxNumberOfMessages=1,WaitTimeSeconds=3)
    print("Iteration {0} of polling SQS. Got {1} messages this time.".format(
        iteration,len(messages)))
    
    for message in messages:
        
        # Simulate processing the message by printing    
        print ('\nGot Message!\nMessage id: {0} \nMD5 of body - {1}\nMessage Body {2}\nProcessed!\n'.format(
            message.message_id, message.md5_of_body, message.body))
        

        message.delete() # delete the message so no one else gets the work item
    


# Batch operations

You can send up to 10 messages in a single batch.

In [ ]:
response = fifo.send_messages(Entries=[
    {
        'Id': '1', # Identifier of message within a batch
        'MessageBody': 'Saving API calls',
        'MessageGroupId': 'Group1'
    },
    {
        'Id': '2',
        'MessageBody': 'by putting up to 10 messages',
        'MessageGroupId': 'Group1'

    }, {
        'Id': '3',
        'MessageBody': 'in a single',
        'MessageGroupId': 'Group1'
    },
    {
        'Id': '4',
        'MessageBody': 'operation!',
        'MessageGroupId': 'Group1'
    }

])


print(json.dumps(response, indent=2))



### Cleaning up the Queues

You can clean up the queues. You'll want to apply least privelege so that that Producers or Consumers can't perform these types of admin actions in production.

In [ ]:
# Deleting Queues
queue.delete()
fifo.delete()

### Next Steps
Find out how to allow S3 events to get placed on SQS queues
http://docs.aws.amazon.com/AmazonS3/latest/dev/ways-to-add-notification-config-to-bucket.html